In [1]:
from xml.dom import minidom

#from usrinput import trajTofig
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 10.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

from math import *
import numpy as np

import cv2
import sklearn
from sklearn.metrics import confusion_matrix
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, normalization
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import History 
from keras.optimizers import SGD,Adam,RMSprop
from time import time 
from keras.regularizers import l2

Using TensorFlow backend.


In [4]:
def trajTofig(mouseSeries):
    N,_ = mouseSeries.shape
    xmin,ymin = np.floor(np.min(mouseSeries, axis = 0))
    xmax,ymax = np.floor(np.max(mouseSeries, axis = 0))
    imgWidth = np.int_(np.max([ymax-ymin,xmax-xmin]))
    widthtempX = np.floor((imgWidth+20)/2.0-(xmax-xmin)/2.0)
    widthtempY = np.floor((imgWidth+20)/2.0-(ymax-ymin)/2.0)
    imgtemp = np.ones((imgWidth+20,imgWidth+20))

    for i in range(N):
        imgtemp[mouseSeries[i,1]-ymin+widthtempY-3:mouseSeries[i,1]-ymin+widthtempY+3,
                mouseSeries[i,0]-xmin+widthtempX-3:mouseSeries[i,0]-xmin+widthtempX+3]=0.0
    img = cv2.resize(imgtemp,(28,28))
    return img

def interpoint(points):
    N,_ = points.shape
    ptout = []
    for i in range(N-1):
        ptout.append(points[i,:])
        for t in range(30):
            ptout.append(points[i,:]+(t+1)*(points[i+1,:]-points[i,:])/30.0)
    ptout = np.array(ptout)
    return ptout

users = ['s01 (pilot)','s02','s03','s04','s05','s06','s07','s08','s09','s10','s11']
cates = ['caret','check','circle','delete_mark','left_curly_brace','left_sq_bracket',
           'pigtail','question_mark','rectangle','right_curly_brace','right_sq_bracket','star','triangle','v',
          'x','arrow']
teams = ["%02d"% n for n in range(1,11)]
speeds = ['/fast/','/medium/','/slow/']
img = []
ydata = []
trajtemp =[]
for user in users:
    print user
    for cate in cates:
        ydata.append(np.ones((30,))*cates.index(cate))
        for speed in speeds:
            for team in teams:
                filename = './xml_logs/'+user+speed+cate+team+'.xml'
                xmldoc = minidom.parse(filename)
                itemlist = xmldoc.getElementsByTagName('Point')
                pointNum = len(itemlist)
                PtSeries = []
                for i in range(pointNum):
                    PtSeries.append([itemlist[i].attributes['X'].value,itemlist[i].attributes['Y'].value])
                ptSeriest = np.int_(np.array(PtSeries))
                ptSeries = interpoint(ptSeriest)
                trajtemp.append(ptSeriest)
                img.append(trajTofig(ptSeries))
imgdata = np.array(img).reshape((-1,1,28,28))
ydata = np.array(ydata).reshape((-1,))
print imgdata.shape
print ydata.shape

np.save('./onedollardata/imgdata',imgdata)
np.save('./onedollardata/label',ydata)
np.save('./onedollardata/trajdata',trajtemp)


                



s01 (pilot)


/home/jj/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:12: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


s02
s03
s04
s05
s06
s07
s08
s09
s10
s11
(5280, 1, 28, 28)
(5280,)
